In [1]:
import yfinance as yf
import pandas as pd
import os

# Define a dictionary with indices and their corresponding ticker symbols
indices = {
    'S&P 500': '^GSPC',
    'Gold': 'GC=F',
    'ICE U.S. Dollar Index': '^NYICDX',
    'WTI Crude Oil': 'CL=F',
    'VIX index': '^VIX'
}

# Initialize a dictionary to store individual DataFrames
dataframes = {}

# Loop over each index to download data, process it, save as CSV, and prepare for merging
for index_name, ticker in indices.items():
    # Define the filename
    filename = f"{index_name.replace(' ', '_')}_closing_price_interpolated.csv"
    
    # Download data for the index
    try:
        data = yf.download(ticker, start='1927-12-30', end='2024-10-28')[['Close']].reset_index()
        data.columns = ['Date', 'Close_Price']

        # Convert date to datetime, set as index, and resample
        data = data.assign(Date=pd.to_datetime(data['Date'])).set_index('Date').resample('D').mean()

        # Interpolate missing values
        data['Close_Price'] = data['Close_Price'].interpolate(method='linear')

        # Reset index and save to CSV
        data.reset_index().to_csv(filename, index=False)
        print(f"File created: {filename}")

        # Rename the "Close_Price" column to the index name and store in dictionary
        df = data.reset_index()
        df = df.rename(columns={'Close_Price': index_name})
        dataframes[index_name] = df

    except Exception as e:
        print(f"Error processing {index_name}: {e}")

# Check if we have enough DataFrames to merge
if not dataframes:
    print("No data was successfully processed. Exiting.")
else:
    # Merge all DataFrames on the "Date" column
    merged_df = None
    for index_name, df in dataframes.items():
        if merged_df is None:
            merged_df = df  # Start with the first DataFrame
        else:
            merged_df = pd.merge(merged_df, df, on='Date', how='inner')  # Merge subsequent DataFrames
    
    # Save the merged DataFrame to a new CSV file
    merged_filename = "combined_data.csv"
    merged_df.to_csv(merged_filename, index=False)
    print(f"Merged data saved to {merged_filename}.")

print(merged_df)  # Display the last 10 rows of the merged DataFrame

[*********************100%%**********************]  1 of 1 completed


File created: S&P_500_closing_price_interpolated.csv


[*********************100%%**********************]  1 of 1 completed


File created: Gold_closing_price_interpolated.csv


[*********************100%%**********************]  1 of 1 completed


File created: ICE_U.S._Dollar_Index_closing_price_interpolated.csv


[*********************100%%**********************]  1 of 1 completed


File created: WTI_Crude_Oil_closing_price_interpolated.csv


[*********************100%%**********************]  1 of 1 completed

File created: VIX_index_closing_price_interpolated.csv
Merged data saved to combined_data.csv.
           Date      S&P 500         Gold  ICE U.S. Dollar Index  \
0    2000-08-30  1502.589966   273.899994             112.139999   
1    2000-08-31  1517.680054   278.299988             112.599998   
2    2000-09-01  1520.770020   277.000000             111.419998   
3    2000-09-02  1517.347504   276.699997             111.667500   
4    2000-09-03  1513.924988   276.399994             111.915001   
...         ...          ...          ...                    ...   
8818 2024-10-21  5853.979980  2723.100098             104.010002   
8819 2024-10-22  5851.200195  2744.199951             104.080002   
8820 2024-10-23  5797.419922  2714.399902             104.430000   
8821 2024-10-24  5809.859863  2734.899902             104.059998   
8822 2024-10-25  5808.120117  2740.899902             104.260002   

      WTI Crude Oil  VIX index  
0         33.400002  17.690001  
1         33.099998  1

In [2]:
import requests
import pandas as pd

# URL of the Excel file to download
url = "https://markets.newyorkfed.org/read?startDt=2016-03-01&endDt=2024-10-25&eventCodes=500&productCode=50&sort=postDt:-1,eventCode:1&format=xlsx"

# Send a GET request to download the Excel file
response = requests.get(url)

# Specify the local file path to save the downloaded Excel file
file_path = "../EFFR_data.xlsx"

# Save the Excel file locally
with open(file_path, 'wb') as f:
    f.write(response.content)

# Read the Excel file using pandas with the 'xlsxwriter' engine
df_EFFR = pd.read_excel(file_path)

# Convert 'Effective Date' column in df_EFFR to datetime
df_EFFR['Effective Date'] = pd.to_datetime(df_EFFR['Effective Date'])

# Select only the 'Effective Date' and 'Rate (%)' columns from df_EFFR
df_EFFR = df_EFFR[['Effective Date', 'Rate (%)']]

# Set the 'Effective Date' column as the index of df_EFFR
df_EFFR.set_index('Effective Date', inplace=True)

# Merge the "Rate (%)" column from df_EFFR to merged_df_step1 based on the date column
merged_df = merged_df.merge(df_EFFR, how='left', left_on='Date', right_index=True)

# Remove rows with NA values in the 'Rate (%)' column
merged_df = merged_df.dropna(subset=['Rate (%)'])

# Interpolate all columns in merged_df to fill in missing values
merged_df = merged_df.set_index('Date')
merged_df = merged_df.resample('D').interpolate(method='linear')
merged_df = merged_df.reset_index()
merged_df = merged_df.rename(columns={'index': 'Date'})

# Print the first few rows to verify the merge
print(merged_df)

/opt/network/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


           Date      S&P 500         Gold  ICE U.S. Dollar Index  \
0    2016-03-01  1978.349976  1230.300049              98.349998   
1    2016-03-02  1986.449951  1241.099976              98.209999   
2    2016-03-03  1993.400024  1257.400024              97.570000   
3    2016-03-04  1999.989990  1269.900024              97.339996   
4    2016-03-05  2000.579997  1267.666667              97.249997   
...         ...          ...          ...                    ...   
3156 2024-10-21  5853.979980  2723.100098             104.010002   
3157 2024-10-22  5851.200195  2744.199951             104.080002   
3158 2024-10-23  5797.419922  2714.399902             104.430000   
3159 2024-10-24  5809.859863  2734.899902             104.059998   
3160 2024-10-25  5808.120117  2740.899902             104.260002   

      WTI Crude Oil  VIX index  Rate (%)  
0         34.400002  17.700001      0.36  
1         34.660000  17.090000      0.37  
2         34.570000  16.700001      0.37  
3         3

In [3]:
# Data processing
from darts import TimeSeries
import pandas as pd
import numpy as np

# Function to calculate moving averages
def calculate_moving_averages(df, window):
    return df.rolling(window=window).mean()

# Calculate 3-day and 6-day moving averages for all columns except 'Date'
for col in merged_df.columns:
    if col not in ['Date']:
        merged_df[col + '_3D_MA'] = calculate_moving_averages(merged_df[col], window=3)
        merged_df[col + '_6D_MA'] = calculate_moving_averages(merged_df[col], window=6)

# Convert 'Date' column to datetime type and set it as index
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df.set_index('Date', inplace=True)

# Convert moving averages to binary variables
for col in merged_df.columns:
    if '_3D_MA' in col:
        base_col = col.replace('_3D_MA', '')
        merged_df[base_col + '_Binary'] = np.where(merged_df[col] > merged_df[base_col + '_6D_MA'], 1, 0)

# Remove original columns and moving average columns
columns_to_drop = [col for col in merged_df.columns if not col.endswith('_Binary')]
merged_df = merged_df.drop(columns=columns_to_drop)

# Print the updated DataFrame
print(merged_df.head(10))

            S&P 500_Binary  Gold_Binary  ICE U.S. Dollar Index_Binary  \
Date                                                                    
2016-03-01               0            0                             0   
2016-03-02               0            0                             0   
2016-03-03               0            0                             0   
2016-03-04               0            0                             0   
2016-03-05               0            0                             0   
2016-03-06               1            1                             0   
2016-03-07               1            1                             0   
2016-03-08               0            0                             0   
2016-03-09               0            0                             0   
2016-03-10               0            0                             0   

            WTI Crude Oil_Binary  VIX index_Binary  Rate (%)_Binary  
Date                                                 

In [7]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV

# Step 1: Data preprocessing
# Assume merged_df is your DataFrame with the binary target variable and features
# Assume 'S&P 500_Binary' is your target variable and the rest are features
# Prepare the data
X = merged_df.drop(columns=['S&P 500_Binary'])
y = merged_df['S&P 500_Binary']

# Step 2: Split the data
train_size = 0.8
val_size = 0.1
test_size = 0.1
train_end = int(train_size * len(X))
val_end = train_end + int(val_size * len(X))
X_train_val, X_test = X[:val_end], X[val_end:]
y_train_val, y_test = y[:val_end], y[val_end:]
X_train, X_val = X_train_val[:train_end], X_train_val[train_end:]
y_train, y_val = y_train_val[:train_end], y_train_val[train_end:]

# Step 3: Define the walk-forward split window size and step size
window_size = 50  # Fixed training window size of 50
step_size = 50  # Step size of 50
best_error = float('inf')  # Initialize best_error to track the lowest log loss
best_params = None

# Step 4: Walk-forward split (train on first 50, test on next 50, and slide forward)
n_samples = len(X_train)
for start in range(0, n_samples - window_size, step_size):
    end = start + window_size
    test_end = end + step_size  # 50 samples ahead for testing
    if test_end > n_samples:
        test_end = n_samples  # Ensure we don't go beyond the dataset

    # Define training and testing data
    X_train_fold, X_val_fold = X_train.iloc[start:end], X_val
    y_train_fold, y_val_fold = y_train.iloc[start:end], y_val

    # Check if y_val_fold contains both classes
    if len(y_val_fold.unique()) < 2:
        print(f"Skipping fold starting at index {start} due to insufficient class labels in y_val_fold.")
        continue

    # Step 5: Hyperparameter tuning using GridSearchCV
    dt_model = DecisionTreeClassifier()
    param_grid = {
        'max_depth': [10, 20, 30, 50, 70, None],
        'min_samples_split': [10, 18, 20, 22, 25, 40],
        'min_samples_leaf': [3, 4, 5, 6, 10, 20, 30]
    }
    grid_search = GridSearchCV(dt_model, param_grid, cv=None, n_jobs=-1, verbose=0, scoring='neg_log_loss', error_score='raise')
    try:
        grid_search.fit(X_train_fold, y_train_fold)
    except Exception as e:
        print(f"Error occurred in fold starting at index {start}: {e}")
        continue

    # Step 6: Train Decision Tree model with best parameters from GridSearchCV
    best_params_fold = grid_search.best_params_
    dt_model.set_params(**best_params_fold)
    dt_model.fit(X_train_fold, y_train_fold)

    # Step 7: Evaluate the model on the validation set
    y_val_pred_proba = dt_model.predict_proba(X_val_fold)[:, 1]
    error = log_loss(y_val_fold, y_val_pred_proba, labels=[0, 1])

    # Track best error and parameters
    if error < best_error:
        best_error = error
        best_params = best_params_fold  # Store the best parameters
    print(f"Log Loss for fold starting at index {start}: {error:.4f}")

# Step 8: Final evaluation after all folds
print(f"\nBest Log Loss: {best_error:.4f}")
print(f"Best Parameters: {best_params}")

Log Loss for fold starting at index 0: 2.2442
Log Loss for fold starting at index 50: 0.6420
Log Loss for fold starting at index 100: 7.2565
Log Loss for fold starting at index 150: 2.3940
Log Loss for fold starting at index 200: 0.5753
Log Loss for fold starting at index 250: 0.7290
Log Loss for fold starting at index 300: 0.6020
Log Loss for fold starting at index 350: 0.7018
Log Loss for fold starting at index 400: 0.6816
Log Loss for fold starting at index 450: 0.5768
Log Loss for fold starting at index 500: 0.5741
Log Loss for fold starting at index 550: 8.6017
Log Loss for fold starting at index 600: 0.5905
Log Loss for fold starting at index 650: 4.8743
Log Loss for fold starting at index 700: 0.6208
Log Loss for fold starting at index 750: 0.6879
Log Loss for fold starting at index 800: 0.7585
Log Loss for fold starting at index 850: 0.7368
Log Loss for fold starting at index 900: 4.0670
Log Loss for fold starting at index 950: 0.7777
Log Loss for fold starting at index 1000: 0

In [8]:
# Final evaluation on the testing set (using the best model)
final_model = DecisionTreeClassifier(**best_params)  # Train final model with the best parameters
final_model.fit(X_train, y_train)
y_pred_final = final_model.predict(X_test)
y_pred_proba_final = final_model.predict_proba(X_test)[:, 1]
print("\nFinal Classification Report:")
print(classification_report(y_test, y_pred_final))
print("\nFinal Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_final))

# Calculate and print AUC
auc_final = roc_auc_score(y_test, y_pred_proba_final)
print(f"\nArea Under Curve (AUC): {auc_final:.2f}")


Final Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.81      0.66       107
           1       0.88      0.68      0.76       210

    accuracy                           0.72       317
   macro avg       0.72      0.74      0.71       317
weighted avg       0.77      0.72      0.73       317


Final Confusion Matrix:
[[ 87  20]
 [ 68 142]]

Area Under Curve (AUC): 0.78
